Samanalie Perera, 300486075

# Assignment 3: Programming 

In [1]:
from SimPy.Simulation import *
import random
import numpy as np
import math

**1. Modify the q3.py simulation model of an M/M/c queueing system (to create q5.py) as follows.**

- Set up a list m of K monitors, stored as a class variable in the G class. For k ∈ {0, 1, . . . , K −1}, let m[k] monitor the proportion of time that there are k customers in the system.


- Monitor the Resource so as to obtain the performance measures WQ and LQ. See Lectures on SimPy §4.6 (pages 18–19) and §7.7 (page 36).


Test your simulation model with c = 1, λ = 2, μ = 3 and K = 10. For each performance measure, produce a both a point estimate and a 95% confidence interval from 50 replications.


Compare your results to the expected values of 

$$
p = \frac{λ}{μ}\\
LQ = \frac{p^2}{1-p}\\
WQ = \frac{LQ}{λ}\\
π0 = 1 − ρ
πk = ρk(1 − ρ) for k ≥ 1
$$

Comment on your results, especially on whether the expected values are contained within the 95% confidence intervals.


In [2]:
## Useful extras ----------
def conf(L):
    """confidence interval"""
    lower = np.mean(L) - 1.96*np.std(L)/math.sqrt(len(L))
    upper = np.mean(L) + 1.96*np.std(L)/math.sqrt(len(L))
    return (lower, upper)


class Source(Process):
    
    def run(self, N, lam, mu):
        for i in range(N):
            a = Arrival(str(i))
            activate(a, a.run(mu))
            t = random.expovariate(lam)
            yield hold, self, t

class Arrival(Process):
    n = 0 
    
    def run(self, mu):
        Arrival.n +=1 
        arrivalTime = now()
        G.numbermon.observe(Arrival.n)
        
        for j in range(K):
            if(j != Arrival.n):
                G.m[j].observe(0)
        if(Arrival.n < K):
            G.m[Arrival.n].observe(1)
        yield request, self, G.server
                
        t = random.expovariate(mu)
        yield hold, self, t
        yield release, self, G.server 
        Arrival.n -= 1
        G.numbermon.observe(Arrival.n)
        delay = now() - arrivalTime
        G.delaymon.observe(delay)


            #Observing the amount of customers in system 
        for j in range(K):
            if (j != Arrival.n):
                G.m[j].observe(0)
        if(Arrival.n < K):
            G.m[Arrival.n].observe(1)
        
class G:
    server = 'dummy'
    delaymon = 'monitor'
    numbermon = 'monitor'
    m = 'list of monitors'

def models (s, N, lamb, mu, maxtime, rvseed):
        #setting it up 
    initialize()
    random.seed(rvseed)
    G.server = Resource(s, monitored = True)
    G.delaymon = Monitor()
    G.numbermon = Monitor()
        
    G.m = []
    for i in range(K):
        G.m.append(Monitor())
            
        #starting the simulation
    s = Source('Source')
    activate(s, s.run(N, lamb, mu))
    simulate(until = maxtime)
        
        #performance measures 
    W = G.delaymon.mean()
    L = G.numbermon.timeAverage()
    P = []
    for j in range (K):
        P.append(G.m[j].timeAverage())
    LQ = G.server.waitMon.timeAverage()
    lambeff = L/W
    WQ = LQ/lambeff
            
    return(WQ,LQ,P)
        #running the experiment 
K = 10
allWQ = []
allLQ = []
allP = []
for j in range(K):
    allP.append([])
for k in range(50):
    seed = 123*k
    result = models(s = 1,
                    N = 10000,
                    lamb = 2.0,
                    mu = 3.0, 
                    maxtime = 2000000, 
                    rvseed = seed)
    allWQ.append(result[0])
    allLQ.append(result[1])
    tempP = result[2]
    for j in range(K):
        tester = allP[j]
        test = tempP[j]
        tester.append(test)


a = np.mean(allWQ)
(b,c) = conf(allWQ)
rho = 2.0/3.0
LQ = (rho**2)/(1-rho)
WQ = LQ/2.0
print("WQ: expect =%6.4f point=%6.4f ci = (%6.4f, %6.4f)"%(WQ,a,b,c))
        
a = np.mean(allLQ)
(b,c) = conf(allLQ)

print("LQ: expect =%6.4f point=%6.4f ci = (%6.4f, %6.4f)"%(LQ,a,b,c))

p = []
for j in range(K):
    p.append(math.pow(rho, j)*(1-rho))
for j in range(K):
    a = np.mean(allP[j])
    (b,c) = conf(allP[j])
    print("pi[%1d]: expect =%6.4f point=%6.4f ci = (%6.4f, %6.4f)"%(j, p[j],a,b,c))
    

WQ: expect =0.6667 point=0.6677 ci = (0.6566, 0.6787)
LQ: expect =1.3333 point=1.3360 ci = (1.3120, 1.3599)
pi[0]: expect =0.3333 point=0.3341 ci = (0.3315, 0.3367)
pi[1]: expect =0.2222 point=0.2219 ci = (0.2206, 0.2232)
pi[2]: expect =0.1481 point=0.1474 ci = (0.1464, 0.1484)
pi[3]: expect =0.0988 point=0.0982 ci = (0.0971, 0.0993)
pi[4]: expect =0.0658 point=0.0660 ci = (0.0651, 0.0669)
pi[5]: expect =0.0439 point=0.0438 ci = (0.0430, 0.0445)
pi[6]: expect =0.0293 point=0.0294 ci = (0.0288, 0.0300)
pi[7]: expect =0.0195 point=0.0199 ci = (0.0194, 0.0205)
pi[8]: expect =0.0130 point=0.0132 ci = (0.0127, 0.0136)
pi[9]: expect =0.0087 point=0.0088 ci = (0.0083, 0.0092)


**2a. Denote by τ the service time in a single-server queueing system, where τ has a Coxian distribution with the following pictorial representation**

**2ai. Write a coxian() function in Python which generates a random variate from the particular Coxian distribution above.**


In [3]:
def tablelookup(P):
    u = random.random()
    sumP = 0.0
    for i in range(len(P)):
        sumP += P[i]
        if u < sumP:
            return i
    return len(P)

def phase(beta, mu, T):
    i = tablelookup(beta)
    result = 0.0
    n = len(mu)
    while i != n:
        result += random.expovariate(mu[i])
        i = tablelookup(T[i])
    return result
## Useful extras ----------
def conf(L):
    """confidence interval"""
    lower = np.mean(L) - 1.96*np.std(L)/math.sqrt(len(L))
    upper = np.mean(L) + 1.96*np.std(L)/math.sqrt(len(L))
    return (lower, upper)
    
def estimate():
    W = []
    W2 = []
    for i in range(10000):
        x = phase(beta, mu, T)
        W.append(x)
        W2.append(x**2)
    return np.mean(W), np.mean(W2)
beta = [1.0, 0, 0, 0]
mu = [2.0, 3.0]
T = [[0, 0.8],
     [1, 0]]

random.seed(123)
L = []
L2 = []
for j in range(50):
    r = estimate()
    L.append(r[0])
    L2.append(r[1])
print("Conf int = ", conf(L))
print("Conf int 2 = ", conf(L2))   
    

Conf int =  (3.830277689579539, 3.850587675035664)
Conf int 2 =  (30.189490521278454, 30.535115597832128)


ii. Simulate 10000 random variates from this particular Coxian distribution and give a 95% confidence interval for E[τ ] and E[τ 2]. Compare your results to the exact values of E(τ) = 23/30 and E(τ 2) = 17/18 and comment on your findings.

b. Modify a suitable simulation model of an M/G/1 queueing system so that the service time τ is as given in part (a) and λ = 1. Your should import coxian.py from part (a) or reuse its code. Ensure that you use sensible variable names and do not have parameters that are not used in your model.
Determine whether the Pollaczek-Khinchin formula appears to hold by forming a 95% confidence interval for Y = wQ − λ E[τ2] 2(1 − ρ)
from 50 replications, where each replication gives one estimate of Y . Comment on your results.

In [4]:
## Erlang distribution ----------
def erlangvariate(k, mu):
    result = 0.0
    for i in range(k):
        result += random.expovariate(mu)
    return result

class Source(Process):
    def run(self, N, lamb, k, mu):
        for i in range(N):
            a = Arrival(str(i))
            activate(a, a.run(k, mu))
            t = random.expovariate(lamb)
            yield hold, self, t
        
class Arrival(Process):
    """an arrival"""
    def run(self, k, mu):
        arrivetime = now()
        yield request, self, G.server
        t = erlangvariate(k,mu)
        G.servicemon.observe(t)
        G.servicesquaredmon.observe(t**2)
        yield hold, self, t
        yield release, self, G.server
        delay = now()-arrivetime
        G.delaymon.observe(delay)
        
class G:
    server = 'dummy'
    delaymon = 'Monitor'
    servicemon = 'Monitor'
    servicesquaredmon = 'Monitor'

def model(c, N, lamb, k, mu, maxtime, rvseed):
    # setup
    initialize()
    random.seed(rvseed)
    G.server = Resource(c)
    G.delaymon = Monitor()
    G.servicemon = Monitor()
    G.servicesquaredmon = Monitor()
    # simulate
    s = Source('Source')
    activate(s, s.run(N, lamb, k, mu))
    simulate(until=maxtime)
    # gather performance measures
    W = G.delaymon.mean()
    S = G.servicemon.mean()
    S2 = G.servicesquaredmon.mean()
    print("Service times:\n", G.servicemon.yseries())
    print("Service times squared:\n", G.servicesquaredmon.yseries())
    return W, S, S2

## Experiment ----------------
W, S, S2 = model(c=1, N=30, lamb=3, k=2, mu=10, maxtime=2000000, rvseed=123)
print("")
print("Estimate of W:", W)
print("Estimate of S:", S)
print("Estimate of S^2:", S2)





Service times:
 (0.06141925443818422, 0.08072068298398319, 0.23148915446894425, 0.08174670665640571, 0.05748620930082159, 0.09827457314816287, 0.2952873764067431, 0.24422765952063172, 0.11602499967195132, 0.32836066732166724, 0.11705570996467347, 0.2406074168069659, 0.19744399927343295, 0.2171753419351639, 0.28739689656094314, 0.05015977987476573, 0.19160945860647904, 0.1259643866527879, 0.05723699179858774, 0.12609989216034093, 0.10820031212126002, 0.10195168475906521, 0.3684716954455296, 0.4320108532755533, 0.09270383828342971, 0.5410935744664773, 0.035936183755936, 0.07512958135128789, 0.06603288201295375, 0.05032547876320646)
Service times squared:
 (0.0037723248157424116, 0.006515828661400713, 0.05358722863674673, 0.006682524049168446, 0.003304664259777867, 0.009657891727453615, 0.08719463466517757, 0.05964714967492561, 0.013461800548876303, 0.10782072784393062, 0.013702039235333757, 0.057891929022521015, 0.038984132849087395, 0.047165129144655364, 0.08259697615286145, 0.002516003